# DEMO

You will find here two examples of code allowing you to process your **saccade data** but also your **smooth pursuit data** from your raw data

## 1. init

-----------------------------

In [1]:
import ANEMO # import the ANEMO package

#------------------------------------------------------------------------------
# Parameters you can change
#------------------------------------------------------------------------------
RawDatadirpath = 'data' # Raw data directory path
Datadirpath = 'dataDEMO' # Processed data directory path
sub = '001' # Participant identifier
task = None # Name of the Task
ses = '1' # Name of the Session
acq = None # Name of the Aquisition
run = None # IndexRun

#------------------------------------------------------------------------------
# Initialization
#------------------------------------------------------------------------------
ANEMO.init(RawDatadirpath=RawDatadirpath, Datadirpath=Datadirpath, sub=sub,
           task=task, ses=ses, acq=acq, run=run)

## 2. Processing Saccades

-----------------------------

In [2]:
import ANEMO

Datadirpath = 'dataDEMO'

PreProcessing = ANEMO.PreProcessing(Datadirpath).Data
ProcessingSaccades = ANEMO.ProcessingSaccades(Datadirpath).Data

# Parameters common to all functions you can change:
#------------------------------------------------------------------------------
expnames = None # Name of the particular experience to be transformed
                # if None all experiences will be transformed
trials = None # Number or list of the trial to be transformed
              # if None, all the trials will be transformed
recalculate=False # Allows you to indicate if you want to force the calculation
                  # on the trials already processed
#------------------------------------------------------------------------------

###############################################################################
# 1. Transforming your raw data into degrees
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
events_start = 'StimulusOff' # Event marking the start of the trial
before_sacc = 5 # Time to delete before saccades
after_sacc = 15 # Time to delete after saccades
#------------------------------------------------------------------------------

PreProcessing.to_deg(xname='xpr', toxname='xdeg', events_start=events_start,
                     before_sacc=before_sacc, after_sacc=after_sacc,
                     expnames=expnames, trials=trials, recalculate=recalculate)

###############################################################################
# 2. Filter your data
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
order_filter = 2 # The order of the filter
type_filter = 'lowpass' # The type of filter
cutoff = 30 # The critical frequencies for cutoff of filter
sample_rate = 1000 # Sampling rate of the recording
#------------------------------------------------------------------------------

PreProcessing.filter(xname='xdeg', toxname='xdeg_filter',
                     order_filter=order_filter, type_filter=type_filter,
                     cutoff=cutoff, sample_rate=sample_rate,
                     expnames=expnames, trials=trials,
                     recalculate=recalculate)

###############################################################################
# 3. Extrat saccades from your data set
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
Sevents = 'SSACC' # Name of the start of the event saccades
Eevents = 'ESACC' # Name of the end of the event saccades
add_stime = -5 # Add time at the start of the event saccades
add_etime = 15 # Add time at the end of the event saccades
#------------------------------------------------------------------------------

PreProcessing.extract_events(xname='xdeg', toxname='saccade_deg',
                             stop_event=None, Sevents=Sevents, Eevents=Eevents,
                             add_stime=add_stime, add_etime=add_etime,
                             expnames=expnames, trials=trials,
                             recalculate=recalculate)

###############################################################################
# 4. Extracting the different parameters of saccades
#    by fitting the parameters of a pre-defined model to your data.
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
stime = None # Start time of the fitting
etime = -280 # End time of the fitting
step_fit = 2 # Number of steps for the fit

# Dictionary containing the parameters for the generate_params function
arg_generate_params = dict(do_whitening=False)
#------------------------------------------------------------------------------

ProcessingSaccades.Fit(xname='saccade_deg', toxname='fit_saccade',
                       model=ANEMO.Model.saccade, stime=stime, etime=etime,
                       step_fit=step_fit, 
                       arg_generate_params=arg_generate_params,
                       expnames=expnames, trials=trials,
                       recalculate=recalculate)


calculates xdeg from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates xdeg_filter from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates saccade_deg from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates fit_saccade from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished


## 3. Processing Smooth Pursuit

-----------------------------

In [3]:
import ANEMO

Datadirpath = 'dataDEMO'

PreProcessing = ANEMO.PreProcessing(Datadirpath).Data
ProcessingSaccades = ANEMO.ProcessingSaccades(Datadirpath).Data
ProcessingSmoothPursuit = ANEMO.ProcessingSmoothPursuit(Datadirpath).Data

# Parameters common to all functions you can change:
#------------------------------------------------------------------------------
expnames = None # Name of the particular experience to be transformed
                # if None all experiences will be transformed
trials = None # Number or list of the trial to be transformed
              # if None, all the trials will be transformed
recalculate=False # Allows you to indicate if you want to force the calculation
                  # on the trials already processed
#------------------------------------------------------------------------------


###############################################################################
# 1. Transforming your raw data
#    into velocity in degrees per second
###############################################################################
PreProcessing.to_velocity_deg(xname='xpr', toxname='vx', expnames=expnames,
                              trials=trials, recalculate=recalculate)

PreProcessing.to_velocity_deg(xname='ypr', toxname='vy', expnames=expnames,
                              trials=trials, recalculate=recalculate)

###############################################################################
# 2. Filter your data
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
order_filter = 2 # The order of the filter
type_filter = 'lowpass' # The type of filter
cutoff = 30 # The critical frequencies for cutoff of filter
sample_rate = 1000 # Sampling rate of the recording
#------------------------------------------------------------------------------

PreProcessing.filter(xname='vx', toxname='Vx_filter',
                     order_filter=order_filter, type_filter=type_filter,
                     cutoff=cutoff, sample_rate=sample_rate,
                     expnames=expnames, trials=trials,
                     recalculate=recalculate)

PreProcessing.filter(xname='vy', toxname='Vy_filter',
                     order_filter=order_filter, type_filter=type_filter,
                     cutoff=cutoff, sample_rate=sample_rate,
                     expnames=expnames, trials=trials,
                     recalculate=recalculate)

###############################################################################
# 3. Remove saccades from your data set
#    and remove microsaccades during fixation
###############################################################################

#==============================================================================
# a. Detecting the microsaccades present in your data 
#==============================================================================

# Parameters of the function you can change:
#------------------------------------------------------------------------------
threshold = 5 # Relative velocity threshold
mint = 5 # Minimal saccade duration (ms)
maxt = 100 # Maximal saccade duration (ms)
minstep = 30 # Minimal time interval between two detected saccades (ms)
#------------------------------------------------------------------------------

ProcessingSaccades.detect_misacc(vxname='Vx_filter', vyname='Vy_filter',
                                 toxname='MISACC', threshold=threshold,
                                 mint=mint, maxt=maxt, minstep=minstep,
                                 expnames=expnames, trials=trials,
                                 recalculate=recalculate)

#==============================================================================
# b. Adding a new event to your data
#    to indicate when you want to stop removing microsaccades
#==============================================================================

# Parameters:
#------------------------------------------------------------------------------
eventName = 'TargetOn' # Name of the event marking
                       # the appearance of the target
add_time = 100 # Add time at the end of the appearance of the target
#------------------------------------------------------------------------------

data = ANEMO.Data.open(Datadirpath)
for exp in data.keys():
    data_exp = data[exp] # Dictionary correspond to particular experiment
    events = data_exp['Events'] # Events to particular experiment
    events['Stop_MiSacc'] = events[eventName]+add_time # add new events

ANEMO.Data.save(Datadirpath, data)

#==============================================================================
# c. Removing saccades and then microsaccades
#==============================================================================

# Remove Saccades

# Parameters of the function you can change:
#------------------------------------------------------------------------------
Sevents = 'SSACC' # Name of the start of the event saccades
Eevents = 'ESACC' # Name of the end of the event saccades
add_stime = -5 # Add time at the start of the event saccades
add_etime = 15 # Add time at the end of the event saccades
#------------------------------------------------------------------------------

PreProcessing.remove_events(xname='Vx_filter', toxname='Vx_no_sacc',
                            stop_event=None, Sevents=Sevents, Eevents=Eevents,
                            add_stime=add_stime, add_etime=add_etime,
                            expnames=expnames, trials=trials,
                            recalculate=recalculate)


# Remove Micro-Saccades

# Parameters of the function you can change:
#------------------------------------------------------------------------------
add_stime = 0 # Add time at the start of the event micro-saccades
add_etime = 0 # Add time at the end of the event micro-saccades
#------------------------------------------------------------------------------

PreProcessing.remove_events(xname='Vx_no_sacc', Sevents='SMISACC',
                            Eevents='EMISACC', stop_event='Stop_MiSacc',
                            toxname='Vx_NaN', add_stime=add_stime,
                            add_etime=add_etime, expnames=expnames,
                            trials=trials, recalculate=recalculate)

###############################################################################
# 4. Extracting the different parameters of smooth pursuit
#    with the classical method
###############################################################################
classical_method = ProcessingSmoothPursuit.classical_method

# Parameters common to all functions in the classic method you can change:
#------------------------------------------------------------------------------
eventName_TargetOn = 'TargetOn' # Name of the event marking the appearance of
                                # the target

#==============================================================================
# a. The anticipation velocity
#==============================================================================

# Parameters of the function you can change:
#------------------------------------------------------------------------------
add_stime = -50 # Add time at the start of the event
add_etime = 50 # Add time at the end of the event
#------------------------------------------------------------------------------

classical_method.anticipation(xname='Vx_NaN', toxname='classical_anticipation',
                              add_stime=add_stime, add_etime=add_etime,
                              eventName_TargetOn=eventName_TargetOn,
                              expnames=expnames, trials=trials,
                              recalculate=recalculate)

#==============================================================================
# b. The pursuit latency
#==============================================================================

# Parameters of the function you can change:
#------------------------------------------------------------------------------
w1 = 300 # Size of the window 1 in ms
w2 = 50 # Size of the window 2 in ms
off = 50 # Gap between the two windows
crit = 0.1 # Difference criterion between the two linregress detecting if the
           # pursuit begins
#------------------------------------------------------------------------------

classical_method.latency(xname='Vx_NaN', toxname='classical_latency',
                         w1=w1, w2=w2, off=off, crit=crit,
                         eventName_TargetOn=eventName_TargetOn,
                         expnames=expnames, trials=trials,
                         recalculate=recalculate)

#==============================================================================
# c. The velocity of the steady state of the pursuit
#==============================================================================

# Parameters of the function you can change:
#------------------------------------------------------------------------------
add_stime = 400 # Add time at the start of the event
add_etime = 600 # Add time at the end of the event
#------------------------------------------------------------------------------

classical_method.steady_state(xname='Vx_NaN', toxname='classical_steady_state',
                              add_stime=add_stime, add_etime=add_etime,
                              eventName_TargetOn=eventName_TargetOn,
                              expnames=expnames, trials=trials,
                              recalculate=recalculate)

###############################################################################
# 5. Extracting the different parameters of smooth pursuit
#    by fitting the parameters of a pre-defined model to your data.
###############################################################################

# Parameters of the function you can change:
#------------------------------------------------------------------------------
model = ANEMO.Model.SmoothPursuit.velocity_sigmo # Model equation
stime = None # Start time of the fitting
etime = -280 # End time of the fitting
step_fit = 2 # Number of steps for the fit

# Dictionary containing the parameters for the generate_params function
arg_generate_params = dict(eventName_TargetOn='TargetOn',
                           eventName_StimulusOff='StimulusOff',
                           eventName_dir_target='dir_target',
                           do_whitening=False)
#------------------------------------------------------------------------------

ProcessingSmoothPursuit.Fit(xname='Vx_NaN', toxname='fit_smooth_pursuit',
                            model=model, stime=stime, etime=etime,
                            step_fit=step_fit,
                            arg_generate_params=arg_generate_params,
                            expnames=expnames, trials=trials,
                            recalculate=recalculate)


calculates vx from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates vy from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates Vx_filter from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates Vy_filter from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates MISACC from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates Vx_no_sacc from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates Vx_NaN from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates classical_anticipation from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... 

/home/drone/Bureau/ANEMO2LeRetour/docs/source/ANEMO/ProcessingSmoothPursuit.py:152: RuntimeWarning: Mean of empty slice
  anticipation = np.nanmean(data_anti_x)


finished
calculates classical_latency from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
calculates classical_steady_state from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... 

/home/drone/Bureau/ANEMO2LeRetour/docs/source/ANEMO/ProcessingSmoothPursuit.py:393: RuntimeWarning: Mean of empty slice
  steady_state = abs(np.nanmean(data_steady_x))


finished
calculates fit_smooth_pursuit from the file sub-001_task-aSPEM_ses-1_acq-9_run-4... finished
